In [0]:
import numpy as np
from pele.potentials import LJ
from pele.utils import xyz
from pele.angleaxis import rigidbody
from pele.optimize import lbfgs_py


# read in coordinates from xyz file
ref = xyz.read_xyz(open("TIP4P-16.xyz"))
xyz.write_xyz(open("test.xyz", "w"), coords=ref.coords)
# lookup table for atom masses
mass_lookup = {'O': 16., 'H': 1.}

#ref.coords[:] *= 3


In [0]:

# now define a new rigid body system
rb_sites = []
for atomtype, x, i in zip(ref.atomtypes, ref.coords, xrange(len(ref.atomtypes))):
    # every 3rd atom, define a new reigid molecule
    if i % 3 == 0:
        rb = rigidbody.RigidFragment()
        rb_sites.append(rb)
    rb.add_atom(atomtype, x, mass_lookup[atomtype])

# finalize the rigid body setup
for rb in rb_sites:
    rb.finalize_setup()

# define a new rigid body system
rbsystem = rigidbody.RBSystem()
rbsystem.add_sites(rb_sites)

print len(rbsystem.sites), len(rbsystem.indices)

rbcoords = rbsystem.coords_adapter(np.zeros(len(rbsystem.sites)*6))

for site, com in zip(rbsystem.sites, rbcoords.posRigid):
    com[:] = ref.coords[site.indices[0]] - site.atom_positions[0]




In [0]:

# for simplicity just use a lj potential here
pot = LJ(eps=0.1550, sig=3.1536)
# get the flattened coordinate array
print pot.getEnergy(ref.coords.flatten())
rbpot = rigidbody.RBPotentialWrapper(rbsystem, pot)
print rbpot.getEnergy(rbcoords.coords)
e, g = rbpot.getEnergyGradient(rbcoords.coords)
g_n = rbpot.NumericalDerivative(rbcoords.coords, eps=1e-4)
cg = rbsystem.coords_adapter(g-g_n)
print cg.posRigid
print cg.rotRigid
ret = lbfgs_py(rbcoords.coords, rbpot)
print ret.energy
xyz.write_xyz(open("quenched.xyz", "w"), rbsystem.to_atomistic(
    ret.coords), atomtypes=ref.atomtypes)


In [0]:
print rbsystem.get_masses()
print rbsystem.get_permlist()

In [0]:
db = rbsystem.create_database()

In [0]:
bh = system.get_basinhopping(database=db, outstream=None)
bh.run(400)

In [0]:
print "number of minima", db.number_of_minima()
print "number of transition states", db.number_of_transition_states()

In [0]:
import logging
logger = logging.getLogger("pele.connect")
logger.setLevel("WARNING")

In [0]:
from pele.landscape import ConnectManager
manager = ConnectManager(db)
for i in xrange(20):
    min1, min2 = manager.get_connect_job()
    connect = system.get_double_ended_connect(min1, min2, db, verbosity=-1)
    connect.connect()

In [0]:
print "number of minima", db.number_of_minima()
print "number of transition states", db.number_of_transition_states()

In [0]:
from pele.utils.disconnectivity_graph import DisconnectivityGraph, database2graph

graph = database2graph(db)
dgraph = DisconnectivityGraph(graph)
dgraph.calculate()
dgraph.plot()
import matplotlib.pyplot as plt
%matplotlib inline
# plt.figure(figsize=(30,1))
plt.show()